In [13]:
import os
import cv2
model = "mri_wo_swin"  #2950
TP = ["real_B", "fake_B"]
test_epoch = "/test_latest/"
rootdir = "/mnt/pank/pytorch-CycleGAN-and-pix2pix/results/"
for tp in TP:
    if not os.path.isdir(rootdir+model+test_epoch+tp):
        os.makedirs(rootdir+model+test_epoch+tp)

    img_dir = os.listdir(rootdir+model+test_epoch+"images")
    for i in range(len(img_dir)):
        if img_dir[i][-10:-4] == tp:
            img = cv2.imread(rootdir+model+test_epoch+"images"+"/"+img_dir[i])
            temp = img_dir[i]
            cv2.imwrite(rootdir+model+test_epoch+tp+"/"+temp,img)

In [3]:
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

import cv2
import os
import numpy as np
# import pytorch_fid.fid_score as fid

def evaluation(dir1, dir2):
    img_dir1 = os.listdir(dir1)
    img_dir2 = os.listdir(dir2)

    img_dir1.sort(key=lambda x: x[:-11])
    img_dir2.sort(key=lambda x: x[:-11])
    print('*',img_dir1[0:10])
    print('***',img_dir2[0:10])
    
    mse_value = 0
    psnr_value = mse_value
    ssim_value = mse_value
    
    for i in range(0,len(img_dir1)):
        img1 = cv2.imread(dir1+"/"+img_dir1[i])
        img2 = cv2.imread(dir2+"/"+img_dir2[i])
        mse_value = mse(img1, img2) + mse_value
        psnr_value = psnr(img1, img2, data_range=255) +psnr_value
        ssim_value = ssim(img1, img2, multichannel=True) + ssim_value

    print("MSE = ", mse_value/len(img_dir1))
    print("PSNR = ", psnr_value/len(img_dir1))
    print("SSIM = ", ssim_value/len(img_dir1))

    return mse_value/len(img_dir1), psnr_value/len(img_dir1), ssim_value/len(img_dir1)

model = "test"#"ixi_SwinG_adj_id_0"
test_epoch = "/test_latest/"
rootdir = "/mnt/pank/pytorch-CycleGAN-and-pix2pix/results/"
real_dir = rootdir+model+"/real"
fake_dir = rootdir+model+"/fake"
mse, psnr,ssim = evaluation(real_dir,fake_dir)


* ['6A1_90_real_B.png']
*** ['3A1_90_fake_B.png']
MSE =  247.03922526041666
PSNR =  24.20314444200827
SSIM =  0.827280496776632


In [1]:
import numpy as np
from PIL import Image
import torch
import cv2
import os

model = "mriv4_pix2pix"
test_epoch = "/test_latest/"
rootdir = "/mnt/pank/pytorch-CycleGAN-and-pix2pix/results/"
fake_dir = rootdir+model+test_epoch+"fake_B"
real_dir = rootdir+model+test_epoch+"real_B"
fake = cv2.imread(os.path.join(fake_dir, 'A1_70_fake_B.png'))
real = cv2.imread(os.path.join(real_dir, 'A1_70_real_B.png'))

diff1 = np.sum(fake[:,:,0]-fake[:,:,1])
diff2 = np.sum(real[:,:,0]-real[:,:,1])

dir = "/mnt/pank/pytorch-CycleGAN-and-pix2pix/checkpoints/mriv4_SwinG_neigb_2_id/web/images/epoch006_fake_B.png"
img_repeat = cv2.imread(dir)
diff3 = np.sum(img_repeat[:,:,0]-img_repeat[:,:,1])

In [8]:
import torch
from swinD import Swin_D
import torch.nn as nn
from networks import NLayerDiscriminator

img = torch.zeros([1,3,256,256]) 
net = Swin_D(img_size=256, patch_size=4, in_chans=3, num_classes=1,
                 embed_dim=96, depths=[2, 8, 2], num_heads=[3, 6, 12],
                 window_size=8, mlp_ratio=4., qkv_bias=True, qk_scale=None,
                 drop_rate=0., attn_drop_rate=0., drop_path_rate=0.1,
                 norm_layer=nn.LayerNorm, ape=False, patch_norm=True,
                 use_checkpoint=False)
net1 = NLayerDiscriminator(input_nc=3, n_layers=3)
a = net(img)
b = net1(img)

In [48]:
import cv2
import numpy as np
import os

rootdir = "/mnt/pank/pytorch-CycleGAN-and-pix2pix/results/IXI_SwinG_adj_id_D/test_latest"
dir1 = rootdir+"/real_B"
dir2 = rootdir+"/fake_B"
img_dir1 = os.listdir(dir1)
img_dir2 = os.listdir(dir2)

img_dir1.sort(key=lambda x: x[:-11])
img_dir2.sort(key=lambda x: x[:-11])
best_result1 = []
idx1 = []
for i in range(len(img_dir1)):
    real = cv2.imread(dir1+"/"+img_dir1[i],0)
    fake = cv2.imread(dir2+"/"+img_dir2[i],0)
    h,w = real.shape
    error = np.abs(real - fake)
    k = 0
    for m in range(h):
        for n in range(w):
            if error[m,n] > 200:
                k=k+1
    if k<20000 and k>10000:
        best_result1.append(k)
        idx1.append(img_dir2[i])


In [57]:
a= min(best_result)
idx[23]

'C44_104_fake_B.png'

In [10]:
import torch
import torch.nn as nn
from torch.nn.utils import spectral_norm as sn

features = torch.zeros([1,3,16,16])
features[0,0,0,0] = 1
a = [sn(nn.Conv2d(3, 6, kernel_size=3, stride=2, padding=3)), sn(nn.LeakyReLU(0.2, True))]
b = nn.Sequential(*a)
b(features)

KeyError: 'weight'

In [4]:
import torch
import torchvision

x=torch.zeros([1,3,256,256])
vgg19 = torchvision.models.vgg16(pretrained=True)
chosed = [1,6,11,20,29]
out = x
for i, layer in enumerate(vgg19.features):
    out = layer(out)


Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True

In [6]:
a = True
b = False
a or b

True